# Basic statements

This section covers the basic Python language constructs supported in synthesizable contexts. Other statements like the allowed operators, function calls and the coroutine utilities `async`, `await` and `while` are described in separate notebooks.

---
## if statement

How CoHDL translates if statements depends on their condition:

* condition is a type qualified object

    CoHDL produces a VHDL if statement. This is only possible in sequential contexts.
* otherwise

    All other objects are considered constants. CoHDL translates only the active branch and ignores the other one. This is similar to VHDLs if generate and allowed in all contexts.


In [ ]:
from cohdl import Bit, BitVector, Port, Entity
from cohdl import std

class ExampleIf(Entity):
    select_a = Port.input(Bit)

    a = Port.input(BitVector[4])
    b = Port.input(BitVector[4])

    result = Port.output(BitVector[4])

    def architecture(self):
        invert = True

        @std.sequential
        def proc_select():
            # Since 'invert' is not a Signal, Variable or Temporary this
            # if is evaluated at compile time. Only one branch
            # will show up in the generated VHDL.
            if invert:
                # The condition of the inner if statement is runtime variable
                # and thus translated into a VHDL if statement. This is only
                # possible in sequential contexts.
                if self.select_a:
                    self.result <<= self.a
                else:
                    self.result <<= self.b
            else:
                if self.select_a:
                    self.result <<= self.b
                else:
                    self.result <<= self.a

print(std.VhdlCompiler.to_string(ExampleIf))

---
## if expression

If expressions are supported in both concurrent and sequential contexts. Like the ternary operator (`cond ? a : b`) known from C, they select one of two arguments based on a condition.

In [ ]:
from cohdl import Bit, BitVector, Port, Entity
from cohdl import std

class ExampleIfExpr(Entity):
    select_a = Port.input(Bit)
    select_b = Port.input(Bit)

    a = Port.input(BitVector[4])
    b = Port.input(BitVector[4])
    c = Port.input(BitVector[4])

    result_1 = Port.output(BitVector[4])
    result_2 = Port.output(BitVector[4])

    def architecture(self):
        @std.concurrent
        def logic_single():
            self.result_1 <<= self.a if self.select_a else self.b
        
        @std.sequential
        def logic_nested():
            self.result_2 <<= self.a if self.select_a else self.b if self.select_b else self.c

print(std.VhdlCompiler.to_string(ExampleIfExpr))

---
## select_with

CoHDL provides a magic builtin function `select_with` to choose one of multiple options based on a single selector input.

In [ ]:
from cohdl import select_with, BitVector, Port, Entity
from cohdl import std

class ExampleSelect(Entity):
    selector = Port.input(BitVector[2])

    a = Port.input(BitVector[4])
    b = Port.input(BitVector[4])
    c = Port.input(BitVector[4])
    d = Port.input(BitVector[4])

    result = Port.output(BitVector[4])

    def architecture(self):
        @std.concurrent
        def select_output():
            # choose one of the inputs based on the current state of `selector`
            # and assign it to the result
            # the different inputs are provided as a dictionary
            self.result <<= select_with(
                self.selector,
                {
                    "00": self.a,
                    "01": self.b,
                    "10": self.c,
                    "11": self.d
                },
                # default="0000" # optional default state
            )


print(std.VhdlCompiler.to_string(ExampleSelect))

---
## for loops

For loops are unrolled at compile time.

In [ ]:
from cohdl import BitVector, Port, Entity
from cohdl import std

class ExampleFor(Entity):
    input_a = Port.input(BitVector[4])
    input_b = Port.input(BitVector[4])

    result = Port.output(BitVector[8])

    def architecture(self):
        @std.concurrent
        def example_range():
            for index in range(8):
                if index % 2 == 0:
                    self.result[index] <<= self.input_a[index // 2]
                else:
                    self.result[index] <<= self.input_b[index // 2]


print(std.VhdlCompiler.to_string(ExampleFor))

Since CoHDL is not limited by VHDLs for loop capability, it can support a broad range of iterables including those returned by `enumerate` or `zip`.

In [ ]:
from cohdl import BitVector, Port, Entity
from cohdl import std

class ExampleFor(Entity):
    a = Port.input(Bit)
    b = Port.input(Bit)
    c = Port.input(Bit)

    result = Port.output(BitVector[3])

    def architecture(self):
        inputs = [self.a, self.b, self.c]

        @std.concurrent
        def example_enumerate():
            # iterate over a list containing the three inputs
            # and assign them to the corresponding output bit
            for nr, input in enumerate(inputs):
                self.result[nr] <<= input


print(std.VhdlCompiler.to_string(ExampleFor))

In [ ]:
from cohdl import BitVector, Port, Entity
from cohdl import std

class ExampleFor(Entity):
    a = Port.input(Bit)
    b = Port.input(Bit)
    c = Port.input(Bit)

    result = Port.output(BitVector[3])

    def architecture(self):
        inputs = [self.a, self.b, self.c]

        @std.concurrent
        def example_zip():
            # equivalent to the previous example
            # iterate over pairs of input and output bits
            # instead of using an index
            for output, input in zip(self.result, inputs):
                output <<= input


print(std.VhdlCompiler.to_string(ExampleFor))

---
## for loops - special case

CoHDL defines one special type of for loops. When the loop body consists of a single if-statements that ends in a break, the loop is not simply unrolled but turned into a chain of if-else-statements. This follows the expected Python behavior - only the first iteration with a true condition is executed.

This is the only construct where break statements are allowed in for loops. It is also the only way to use Pythons for-else clause to provide an optional fallback that will be placed in the innermost else branch.

The following code block demonstrates this on a simple entity. However, the initial motivation for this feature where more complex designs. For example a bus interconnect can iterate over a list of connected devices, select one based on an address and perform some device specific operation.

In [ ]:
from cohdl import BitVector, Port, Entity
from cohdl import std

class ExampleFor(Entity):
    input = Port.input(BitVector[4])
    output = Port.output(BitVector[4])

    def architecture(self):
        @std.sequential
        def example_for():
            self.output <<= "0000"

            for nr, inp in enumerate(self.input):
                # set the corresponding output bit
                # for the first high input bit
                if inp:
                    self.output[nr] <<= "1"
                    break
            else:
                # optional else case used when break is encountered (all inputs low)
                self.output <<= "1111"

print(std.VhdlCompiler.to_string(ExampleFor))

---
## assertions

CoHDL attempts to check Python assertions at compile time and generates VHDL assertions if that is not possible

In [ ]:
from cohdl import BitVector, Port, Entity, static_assert
from cohdl import std

class ExampleAssert(Entity):
    input = Port.input(BitVector[4])
    output = Port.output(BitVector[4])

    def architecture(self):
        @std.sequential
        def example_for():
            # assertions with runtime variable conditions are translated into
            # vhdl assertions
            assert self.input != "0110", "runtime assertion"

            # assertions with runtime constant conditions are evaluated by the
            # compiler (similar to C++ static_assert)
            assert self.input.width == 4, "constant assertion ok"

            # alternatively you can use the builtin static_assert
            # that is always evaluated at compile time and never present
            # in the generated vhdl
            static_assert(self.input.width == 4, "constant assertion ok")

print(std.VhdlCompiler.to_string(ExampleAssert))